# Creating a SIP Calling App!
--> (creating own softphone/PBX)
## Pt 1. What is  SIP?

SIP is the standard protocol used to intiate calls. SIP requests can be sent locally, or to a provider, who will forward your request to a phone remotely.

In SIP, we:
- Specify 'to' and 'from' adress
- Register with a server (for them to dispatch calls to public switching telephone network (PSTN))
- Negotiate type of call that will be placed

We will start exploring the fundamentals of SIP, by building a local app that can both receive incoming calls, and make outbound calls, to IP adresses listening for UDP connections on the standardized SIP port. Then, we will send and receive SIP requests from a SIP trunk provider, who is that gateway which sends our SIP requests to PSTN.

We will be abstracting/treating as a black box:
- UDP transport via wifi, and the entire wireless protocol/internet stack

and focusing on the actual message that we send Via this transport, and exactly how it works/what it expects in order to establish/terminate calls both over local internet, and over the public internet. After this, we'll deal with the transport of audio packets.

## Pt.2 Implementing Basic SIP flow

Let's see what a basic SIP request looks like, so we can build it from scratch. First, we will intiate a local call, sending an SIP INVITE from one machine on LAN to another. Usually things are not this easy; we forward the SIP request to some PBX or switching system; but we'll do this for simplicity now.

Here is the full request in text-form: [calls.ipynb](calls.ipynb)

If we do a call from linphone (popular SIP caller), we can see the exact packets sent via UDP using wireshark(analyze-sip-follow)

and here is a useful visual diagram that Wireshark provides:

![callflow](callflow.png)

-------------------------------
### Theory/Bakground
Ok.

So, let's look at this as if we'd wanted to invent a protcol that allows calling to be done over the internet. Bascially, we are using the existing internet infrastrucutre, and IP protocols to 

1. Negotiate a call (what SIP does!)
2. Send audio packets remotely

Fortunately, the internet, and wireless transmission has become so fast that this is feasible, now the standardized way of communication just needs to be defined.

If we were to design a minimal application that leverages IP and internet inferastructure to make calls, we would need to adhere to a couple of things:

- Standardization: the computer needs to know how to read the bytes we send (ie special-byte seperated commands) every time.
- Convey the right amount of information (IE the parameters in our request)

So we could imagine sending a packet of bytes seperated by some special byte in-between each command, that desribes who is calling, and where its going to. (arrows are comments)

```
CALL --> Type of command
Unique_ID: 1023fsndlfjn123 --> Unique call ID
From:<person>@<ip_addr> --> After 'From', read who it is
Audio:1 --> After 'Audio:' read T/F
```
Then we could just create a listener that waits, parses, and responds according to our made-up protocl.

Let's explore how SIP does this.

-----------------------------------



In [ ]:
import socket
import secrets # for branch/tag
'''
ALWAYS starts with type of command:
ALWAYS end each command with \r\n bytes

(this is a minimal example with only
required parameters)
--------------------------------------

INVITE sip:{ip_addr} SIP/2.0
Via: SIP/2.0/UDP {ip_addr}:{port=5060};branch=z9hG4bK.GOXyfZY7y;rport
From: "{DisplayName}" <sip:{username}@{server_ipaddr}>;tag=MDiQjEkoG
To: {"othername}" sip:{dest_addr}
CSeq: 20 INVITE
Call-ID: {unique_call_id=lq66o-0T6P}
Max-Forwards: 70
Contact: <sip:{ip_addr}>
Content-Type: application/sdp
Content-Length: {len(sdp)}

'''

# SDP: for negotiating media 
# more on audio formatting later
# when we discuss audio transport
'''
v=0 // version
o =- {rand1} {rand1} IN IP4 {ip_addr}// origin: empty,session_id,session version,network type,adress type (ip4),origin adress
s=SIP call // name
c =IN IP4 {ip_addr} // connection info
t=0 0 // time info 
m=audio 49177 RTP/AVP 0 // media format, port, type, codece 0

'''

# sip client which builds all request types!
# local (for now)
class SIPClient:
    ########################

    # for tag/branch
    @staticmethod
    def create_ranhex(length=10):
        return secrets.token_hex(length//2)


    #########################
    def __init__(self,ip_addr,username,timeout_retry=2,max_retries=10):
        self.ip = ip_addr # loacl
        self.max_retries = max_retries
        self.username = username
        

        # udp socket to listen/recv
        self.sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
        self.port_src = 5061
        self.sock.bind((self.ip,self.port_src)) # normal port
        self.sock.settimeout(timeout_retry)

    #########################
    def create_sip_invite(self,branch,call_id,tag,dest_addr,session_sdp):
        sdp_info = [
            'v=0',
            f'o=- {session_sdp} {session_sdp} IN IP4 {self.ip}',
            's=sip call',
            f'c=IN IP4 {self.ip}',
            f'm=audio 49922 RTP/AVP 0', # port!
            f'a=rtpmap:0 PCMU/8000', # TODO: COME BACK TO THIS
            '\r\n' # empty line (not appended to w/join)
        ]
        sdp = '\r\n'.join(sdp_info)
        length_sdp = len(sdp) #length in bytes, chars=bytes

        sip_params = [
            f'INVITE sip:{self.ip} SIP/2.0',
            f'Via: SIP/2.0/UDP {self.ip}:{self.port_src};branch=z9hG4bK.{branch};rport',
            f'From: <sip:{self.username}@{self.ip}>;tag={tag}', # leave blank for now...
            f'To: sip:{dest_addr}', #local ip
            f'CSeq: 20 INVITE',
            f'Call-ID: {call_id}',
            f'Max-Forwards: 70',
            f'Contact:<sip:{self.ip};transport=udp>',
            f'Content-type: application/sdp',
            f'Content-length: {length_sdp}',# compute beforehand
            '\r\n' # empty line
        ]
        full_msg = '\r\n'.join(sip_params) + sdp
        return full_msg
    #########################
    # send msg, get (forc_ip to make recvd from dest)
    def send_recv(self,msg,dest,port,force_ip=True):
            self.sock.sendto(msg.encode('utf-8'),(dest,port))

            # try to recv/send!
            retries = 0
            while retries <= self.max_retries:
                try:
                    data,addr= self.sock.recvfrom(4096)
                    if dest == addr[0]:
                        return data,addr
                    else:
                        pass # wrong adress...
                except socket.timeout:
                    retries+=1
                    print(f"timeout, retrying:{retries}")
                    self.sock.sendto(msg.encode('utf-8'),(dest,port))
                    if retries == self.max_retries:
                        raise TimeoutError("Could not send packet")
                    
            


    #########################

    def initiate_call(self,dest_addr,port):
        # INVITE --> (repeat if nec.)
        # TRYING <--
        # RINGING <--
        # ACK (the ok) -->
        # BYE --><---
        print("Call initiating...")

        # generate: branch: z9hG4bK.GOXyfZY7y
        # tag: (for entire)
        # call-id for entire
        branch = SIPClient.create_ranhex()
        tag = SIPClient.create_ranhex()
        call_id = SIPClient.create_ranhex()
        session_sdp = secrets.randbits(32)

        full_invite = self.create_sip_invite(branch=branch,
                                                  dest_addr=dest_addr,
                                                  tag=tag,
                                                  call_id=call_id,
                                                  session_sdp=session_sdp)
        print(full_invite)
        recvd_msg,addr= self.send_recv(full_invite,dest=dest_addr,port=port)
        print(addr)
        print(recvd_msg)
        print("CALLED!")
        return recvd_msg
        
    #########################
    def close_client(self):
        self.sock.close()
        print("sucessfully closed client")


iphone = '100.105.32.112'
imac = '100.124.130.10'
client1 = SIPClient(imac,"Charl")
recvd_resp = client1.initiate_call(imac,port=5060)
client1.close_client()

Call initiating...
INVITE sip:100.124.130.10 SIP/2.0
Via: SIP/2.0/UDP 100.124.130.10:5061;branch=z9hG4bK.d31c718185;rport
From: <sip:Charl@100.124.130.10>;tag=5ca0dbbb3e
To: sip:100.124.130.10
CSeq: 20 INVITE
Call-ID: c8122a1285
Max-Forwards: 70
Contact:<sip:100.124.130.10;transport=udp>
Content-type: application/sdp
Content-length: 140

v=0
o=- 1376374036 1376374036 IN IP4 100.124.130.10
s=sip call
c=IN IP4 100.124.130.10
m=audio 49922 RTP/AVP 0
a=rtpmap:0 PCMU/8000


timeout, retrying:1
timeout, retrying:2
('100.124.130.10', 5060)
b'SIP/2.0 100 Trying\r\nVia: SIP/2.0/UDP 100.124.130.10:5061;branch=z9hG4bK.d31c718185;rport\r\nFrom: <sip:Charl@100.124.130.10>;tag=5ca0dbbb3e\r\nTo: sip:100.124.130.10\r\nCall-ID: c8122a1285\r\nCSeq: 20 INVITE\r\n\r\n'
CALLED!
sucessfully closed client


In [8]:
client1.close_client()

sucessfully closed client


Ok, awesome! Now we have a way to initiate calls, that *actually works*! Its kind of cool to see my phone ring when it gets this request!

But now we need to handle a couple more things to get a basic LAN setup together:
- Acknowledge the 'OK' to our INVITE (dest. answers)/Decline
- Acknowledge the 'BYE' if our client hangs-up, or send our own 'BYE'
- Monitor for responses

This will be pretty similar as our previous functions, we just need to keep some things in mind:
- Copy over all 'tag's & caller ID(they remain for the whole call)
- Branch is for specific parts of the call; 'ACK' and 'BYE' get new branches

> Branches basically identify a specifc part of the call negotiation.

In [16]:
'''ParseSIP
takes all important info about packet and extracts it.

Responses have type of resp at END of first header
'''
def parse_sip_response(self,msg):
    headers = msg.split('\r\n')
    # num and cmd type
    cmd_type = (' ').join(headers[0].split(' ')[-2:])

    # get into a dict list
    pairs = [prt.split(': ') for prt in headers[1:-2]]
    sip_params = {header:value for header,value in pairs}

    # making sure we extract branches
    identifiers = {header:sip_params[header].split(';')[1] for header in sip_params.keys() if any(substring in sip_params[header] for substring in ["branch=","tag="])}
    identifiers["Call-ID"] = sip_params["Call-ID"]

    return cmd_type,sip_params,identifiers


example_sip = recvd_resp.decode('utf-8')

SIPClient.parse_sip_response = parse_sip_response
# Now, ok! We actually don't even need to parse this one; we just
# need to wait until we receive an accept or decline...
print(parse_sip_response(None,example_sip))

('100 Trying', {'Via': 'SIP/2.0/UDP 100.124.130.10:5061;branch=z9hG4bK.8d4a0c144a;rport', 'From': '<sip:Charl@100.124.130.10>;tag=b44d5118a9', 'To': 'sip:100.124.130.10', 'Call-ID': 'a3aedeb264', 'CSeq': '20 INVITE'}, {'Via': 'branch=z9hG4bK.8d4a0c144a', 'From': 'tag=b44d5118a9', 'Call-ID': 'a3aedeb264'})


-------------------------

Ok, now we have the ability to easily parse a request, understanding what type it is, and what the branches are (tags dont change).

Now, I'd like to create a general abstraction for SIP commands that we wish to send, so that:

- Specify type (request/response)
- Add in custom fields that may be necessary
- Use/generate new branch parameters

And in another thread:
- Listen for incoming SIP requests
- Parse request
- pass in a queue for another thread to respond to

So, we have to
1. Create general template for SIP requests/responses
2. Make two threads; one queues/parses, one send appropraite responses (and check we're doing for the right call)
3. Design 'flow' based on whether call is outgoing/inbound (IE: a message is queued, what do we respond with now?)

--------

In [19]:
'''
Design is to have a basic SIP command layed out
with the usual parameters

- Assumed transport is UDP

Given from call instance:
callid,via,branch, tags, from, to, dest

May not need via, if we're responding to a call.
'''
def generate_sip_command(self,type,cmd,cmd_num,dest_ip,identifiers,via=True,extra_cmds=[]):
    full_sip = []
    # difference in titles
    if type == "request":
        sip_title = f"{cmd} sip:{dest_ip} SIP/2.0"
    elif type == "response":
        sip_title = f"SIP/2.0 {cmd_num} cmd"
    full_sip.append(sip_title)

    # via parameter
    via = f"Via: SIP/2.0/UDP"
    if via:
        via += f"{self.from_ip}:{self.port}"
    if "Via" in identifiers.keys() and type == "response":
        via += f";{identifiers['Via']}"
    else:
        via += f";{SIPClient.create_ranhex()}" # a new branch
    full_sip.append(via)
    
    # From header
    from_header = f'From: <sip:{self.username}@{self.ip}>'

    if "From" in identifiers.keys():
        from_header += f";{identifiers['From']}"
    full_sip.append(from_header)
    
    # To header
    to_header = f"To: sip:<{dest_ip}>"
    if "To" in identifiers.keys():
        to_header += f";{identifiers['To']}"
    full_sip.append(to_header)

    # Call-id
    callID = f"Call-ID: {identifiers['Call-ID']}"
    full_sip.append(callID)

    # any extra commands!
    [full_sip.append(header)for header in extra_cmds]

    # with breaks throughout and at end...
    return "\r\n".join(full_sip) + "\r\n"

    
SIPClient.generate_sip_command = generate_sip_command    

I've implemented this so far:
- general SIP template (tracks branches/tags)

and so we'll be able to easily respond to call-requests (INVITE), and also
acknowledge declines, accepts, and hang-ups.

Now, let's create a general algorithm for how our machine handles calls:
- listen for calls, create new thread for each (share port)
    - new call ID + invite = new call instance --> activate ringing/app.
    - Outgoing call-request = new call instance
- Listen for info relating to current calls, respond accordingly...

-------------------

Now, I've created the basic SIP class that allows:
- Parse SIP messages, extract headers, get branch/tags/call id
- Respond appropriately to accept/decline to establish a call!

Now that we've implemented this, we need to learn how to get the *audio* involved
for the call to not immediately hang up.


------------------

## RTP (& SDP)

RTP is the media transport protocol used for VOIP and SIP.

Since we're using UDP which is an 'unreliable' protocol (but fast), RTP is how we send audio packets using the UDP protocol and inferastructure.

Here is the raw information that is sent to communicate between servers:

```
headers:
IP --> UDP --> RTP --> *PAYLOAD*
```

these are all just protocols specifying where, and how information should be sent (we're just leveraging the inferastructure/protocols as before). The *payload* is the actual audio that is sent, and is the most important part. The headers specify in what order the audio should be played, and information about the audio. However before going into this, let's review SDP (how audio is negotiated):

SDP serves the purpose of
- Establishing the type of encoding/audio format during RDP
- Establishing the ports (one port for each user) that audio will be received from

we've already seen SDP, and its pretty easy to understand, so we won't go over it fully here, just know its how we neogtiate what will happen in RTP.



Then, lets implement a web-server/gui that can be acessed from any device, and have these functionalities:

- Multi-call mergign (or seperate windows)
- Noise filtering, combining, ai voice
- 